In [1]:
import syft as sy
import torch as th
import nest_asyncio
from syft.grid.duet.om_signaling_client import register
from syft.grid.duet.webrtc_duet import Duet

nest_asyncio.apply()
sy.VERBOSE = False


def join_duet(target_id):

    signaling_client = register(url="http://ec2-18-191-23-46.us-east-2.compute.amazonaws.com:5000")
    my_domain = sy.Domain(name="Joiner")
    print("♫♫♫ > Duet Client ID: " + signaling_client.duet_id)
    duet = Duet(node=my_domain,
                target_id=target_id,
                signaling_client=signaling_client,
                offer=False)
    
#     begin_duet_client_logger(duet)
    
    return duet

## Create a Signaling Client
This resource can be used / developed by different nodes (network, domain, device, vm) using different types of connection protocols (http, websocket, etc.)
Thus, we abstracted the concept of signaling server, in order to make this process as generic as possible.

In [2]:
duet = join_duet(target_id='1fe27ed437b9ee6e4a9033dff678a934')

♫♫♫ > Duet Client ID: 1237f7a4513d12d0353e3de5a3fdb11d
♫♫♫ > ...using a running event loop...


In [3]:
duet.store[0].request()

Request Message Id:<UID:58bca5f1-65b3-4932-8944-beb55af2830c>


In [24]:
import time
import sys

len(duet.requests.pandas)
r = duet.requests[0]

In [28]:
r.named

'UNNAMED'

In [46]:
duet.store[0].get()

tensor([1, 2, 3])

In [6]:
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet, searchable=True)

In [10]:
p_x.__del__()

In [12]:
import gc

In [15]:
gc.collect()

22

In [19]:
duet.store

[]

In [8]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet, searchable=True)
for i in range(10):
    p_x = p_x + p_x
print("My Result: ", p_x.get())

Send/Request remote operations
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [ ]:
duet.requests

In [77]:
x = th.tensor([1,2,3]).send(duet, searchable=True)

In [78]:
y = th.tensor([1,2,3]).send(duet, searchable=True)

In [79]:
duet.store.pandas

,ID,Tags,Description
0,<UID:45d14975-0e6c-4ff2-8ba7-31f47cd1a72a>,[],
1,<UID:9d76f4e1-62c9-45e1-b4ec-01ad88f1661e>,[],


In [80]:
duet.store['45d14975-0e6c-4ff2-8ba7-31f47cd1a72a'].request()

In [81]:
duet.store['9d76f4e1-62c9-45e1-b4ec-01ad88f1661e'].request()

In [59]:
duet.store[0].request()

In [66]:
duet.store[0].request()

In [51]:
duet.store[0].get()

tensor([1, 2, 3])

In [56]:
duet.store[0].get()

tensor([1, 2, 3])

Send/Request remote operations
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [3]:
signaling_client2 = register(url="http://ec2-18-222-9-218.us-east-2.compute.amazonaws.com:5000")
my_domain2 = sy.Domain(name="Maria2")
print("Please enter ID of node you're connecting to!")
target_id2 = input()

duet2 = Duet(node=my_domain2,
            target_id=target_id2,
            signaling_client=signaling_client2,
            offer=False)

Duet ID:  820ad28ebb50f10356cf155cdd1c68cb
Please enter ID of node you're connecting to!


 ad824e6a7c9c483e138e175ba223bcf7


> Using a running event loop


In [4]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("Pointer: ", p_x)
print("My Result: ", p_x.get())

Send/Request remote operations
Pointer:  <syft.proxy.torch.TensorPointer object at 0x7fe0010af910>
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [5]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet2)
for i in range(10):
    p_x = p_x + p_x
print("Pointer: ", p_x)
print("My Result: ", p_x.get())

Send/Request remote operations
Pointer:  <syft.proxy.torch.TensorPointer object at 0x7fe0010b3a50>
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


Exception in callback Transaction.__retry()
handle: <TimerHandle when=250.704957293 Transaction.__retry()>
Traceback (most recent call last):
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/nest_asyncio.py", line 190, in run
    ctx.run(self._callback, *self._args)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/aioice/stun.py", line 299, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/asyncio/futures.py", line 249, in set_exception
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=(Message(messa...xe4\x17\xaa+'), ('192.168.0.36', 52893))>


## Create Our Domain
Now, we need to create our domain which will work as a server, computing remote requests.

In [4]:
my_domain = sy.Domain(name="Maria")

## Start Duet Process
Now, we can start our ICE protocol, if everything goes well, at the end of this process we'll be connected
with the other side and ready for send/receive messages.

**PS: It is important to pay attention to the "offer" parameter,if this parameter is the same for both peers, communication will not be established.**

In [5]:
duet = Duet(node=my_domain,
            target_id="265745f02c677aabe7fd8a10b0087ab7",
            signaling_client=signaling_client,
            offer=False)

> Using a running event loop


## Send/Receive Requests/Responses
The great advantage of a full-duplex channel is: You can act as a server and as a client simultaneously.

In [6]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("Pointer: ", p_x)
print("My Result: ", p_x.get())

Send/Request remote operations
Pointer:  <syft.proxy.torch.TensorPointer object at 0x7fac80b9c710>
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [ ]:
duet.close()